In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [34]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (323.84 seconds): Collecting URLs... theverge            

n_total     25692
n_unique    25635
dtype: int64

,pubs,actual_pub
https://businessinsider.co.za/automobile-association-south-africa-vehicle-purchases-keeping-vehicles-for-longer-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/dave-king-sanctioned-by-british-takeover-panel-for-rangers-deal-2019-10,"businessinsider, news24",businessinsider
https://cnbc.com/2019/10/11/trump-gives-his-administration-broad-powers-to-put-sanctions-on-turkey.html,"cnbc, realclearpolitics",cnbc
https://cnn.com/2019/10/11/media/fox-news-shepard-smith/index.html,"cnn, axios",cnn
https://hotair.com/archives/ed-morrissey/2019/10/10/piers-morgan-extinction-rebellion-activist-dont-practice-preach,"townhall, hotair",hotair
https://msnbc.com/hallie-jackson/watch/appeals-court-upholds-democrats-subpoena-for-trump-financial-records-71067717523,"msnbc, nbcnews",msnbc
https://msnbc.com/the-beat-with-ari/watch/swizz-beatz-on-working-with-beyonce-jay-z-drake-and-upending-the-art-world-70266437589,"msnbc, nbcnews",msnbc
https://msnbc.com/the-last-word/watch/lawrence-o-donnell-answers-your-impeachment-questions-71107653700,"msnbc, nbcnews",msnbc
https://nationalreview.com/2019/10/the-media-bends-over-backward-to-protect-elizabeth-warren-from-the-washington-free-beacons-damaging-scoop,"realclearpolitics, nationalreview",nationalreview
https://nationalreview.com/2019/10/turkey-and-the-kurds-its-more-complicated-than-you-think,"realclearpolitics, nationalreview",nationalreview


huffpost                  76
cnn                      846
investing.com            962
politico                 211
time                      19
cnbc                     138
fox                      342
bbc                      271
businessinsider          598
morningstar               91
wsj                      107
nytimes                  107
guardian                 166
reuters                 1146
washingtontimes          465
washingtonpost           124
cbs                      407
marketwatch              149
atlantic                 101
vice                     104
npr                      867
newrepublic               20
yahoo                    389
independent              717
heritage                 280
zdnet                    391
townhall                 616
abcnews                  107
hotair                    84
cbc                      289
                        ... 
metro                    227
msnbc                    103
nationalreview           741
news24        

In [35]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [36]:
select_results = crawler.select(); display(*select_results)

100.00% (29.53 seconds): Selecting URLs... cryptonews          

n_total     281
n_unique    281
dtype: int64

,pubs,actual_pub


huffpost                 1
cnn                      1
investing.com           16
politico                 0
time                     0
cnbc                     0
fox                      0
bbc                      4
businessinsider         18
morningstar              3
wsj                      0
nytimes                  0
guardian                 7
reuters                 15
washingtontimes          2
washingtonpost           2
cbs                      0
marketwatch              0
atlantic                 0
vice                     0
npr                      0
newrepublic              0
yahoo                   38
independent              4
heritage                 0
zdnet                    3
townhall                 0
abcnews                  0
hotair                   0
cbc                      0
                        ..
metro                    5
msnbc                    0
nationalreview           0
news24                  14
techcrunch               0
arstechnica              0
e

In [37]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [38]:
download_results = crawler.download(); download_results

100.00% (352.27 seconds): Downloading... euronews            

,downloaded,trashed
abcnews,0,0
afr,0,0
aljazeera,0,6
americanconservative,0,0
arirang,0,0
arstechnica,0,0
arynews,2,2
atlantic,0,0
axios,0,0
bbc,3,1


In [14]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

[]